<a href="https://colab.research.google.com/github/hindanjali/Assignment2_part_a/blob/main/part_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import BatchNormalization

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/nature_12K.zip

In [6]:
train_dir="/content/inaturalist_12K/train"

In [7]:
import pathlib
train_dir = pathlib.Path(train_dir)

In [ ]:
list(train_dir.glob('*/*.jpg'))

In [5]:
image_count = len(list(train_dir.glob('*/*.jpg')))
print(image_count)

9999


In [8]:
test_dir="/content/inaturalist_12K/val"

In [9]:
import pathlib
test_dir = pathlib.Path(test_dir)

In [ ]:
Arachnida = list(test_dir.glob('Arachnida/*'))
Arachnida[:5]

In [ ]:
PIL.Image.open(str(Arachnida[100]))

In [ ]:
Amphibia = list(test_dir.glob('Amphibia/*'))
PIL.Image.open(str(Amphibia[8]))

In [9]:
test_count = len(list(test_dir.glob('*/*.jpg')))
print(test_count)

2000


In [10]:
train_nature_images_dict = {
    'Amphibia': list(train_dir.glob('Amphibia/*')),
    'Animalia': list(train_dir.glob('Animalia/*')),
    'Arachnida': list(train_dir.glob('Arachnida/*')),
    'Aves': list(train_dir.glob('Aves/*')),
    'Fungi': list(train_dir.glob('Fungi/*')),
    'Insecta': list(train_dir.glob('Insecta/*')),
    'Mammalia': list(train_dir.glob('Mammalia/*')),
    'Mollusca': list(train_dir.glob('Mollusca/*')),
    'Plantae': list(train_dir.glob('Plantae/*')),
    'Reptilia': list(train_dir.glob('Reptilia/*')),


}

In [11]:
nature_labels_dict = {
    'Amphibia': 0,
    'Animalia': 1,
    'Arachnida': 2,
    'Aves': 3,
    'Fungi': 4,
    'Insecta': 5,
    'Mammalia': 6,
    'Mollusca': 7,
    'Plantae': 8,
    'Reptilia': 9,

}

In [12]:
x_train, y_train= [], []
for nature, images in train_nature_images_dict.items():
    for image in images:
       # print(image)
        img = cv2.imread(str(image))
        try:
          resized_img = cv2.resize(img,(120,120))#, cv2.INTER_AREA)
        except cv2.error:
          continue
        x_train.append(resized_img)
        y_train.append(nature_labels_dict[nature])
        

In [13]:
test_nature_images_dict = {
    'Amphibia': list(test_dir.glob('Amphibia/*')),
    'Animalia': list(test_dir.glob('Animalia/*')),
    'Arachnida': list(test_dir.glob('Arachnida/*')),
    'Aves': list(test_dir.glob('Aves/*')),
    'Fungi': list(test_dir.glob('Fungi/*')),
    'Insecta': list(test_dir.glob('Insecta/*')),
    'Mammalia': list(test_dir.glob('Mammalia/*')),
    'Mollusca': list(test_dir.glob('Mollusca/*')),
    'Plantae': list(test_dir.glob('Plantae/*')),
    'Reptilia': list(test_dir.glob('Reptilia/*')),

}


In [14]:
x_test, y_test= [], []

for nature, images in test_nature_images_dict.items():
    for image in images:
        #print(image)
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(120,120))
        x_test.append(resized_img)
        y_test.append(nature_labels_dict[nature])

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,test_size=0.10)# random_state=0)
#print(len(x_train), len(x_val),len( y_train),len( y_val))

In [16]:
x_train = np.array(x_train)/255
y_train = np.array(y_train)

In [17]:
x_val = np.array(x_val)/255
y_val = np.array(y_val)

In [18]:
x_test = np.array(x_test)/255
y_test = np.array(y_test)

In [15]:
#x_train = x_train/255
#x_test = x_test/255
#x_val = x_val/255

In [20]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(120, 
                                                            120,
                                                          3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.4),
    
  
    
  ]
)

In [38]:
def cnv(n_filter,s_filter,actv,n_neurons,epochs,dropout,data_aug):

  output_classes = 10
  #n_filter=16
  # s_filter=3
  #actv='relu'
  #n_neurons=128
  if (batch_norm=='yes'):
  if (data_aug == 'yes'):
    model = Sequential([
    data_augmentation,
    layers.Conv2D(n_filter, s_filter, padding='same', activation=actv),
    layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    layers.Conv2D(n_filter, s_filter, padding='same', activation=actv),
    layers.MaxPooling2D(),
    layers.Conv2D(n_filter, s_filter, padding='same', activation=actv),
    layers.MaxPooling2D(),
    layers.Conv2D(n_filter, s_filter, padding='same', activation=actv),
    layers.MaxPooling2D(),
    layers.Conv2D(n_filter, s_filter, padding='same', activation=actv),
    layers.MaxPooling2D(),
    layers.Dropout(dropout),
    layers.Flatten(),
    layers.Dense(n_neurons, activation=actv),
    layers.Dense(output_classes)
    ])
  else:
    model = Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(n_filter, s_filter, padding='same', activation=actv),
    layers.MaxPooling2D(),
    layers.Conv2D(n_filter, s_filter, padding='same', activation=actv),
    layers.MaxPooling2D(),
    layers.Conv2D(n_filter, s_filter, padding='same', activation=actv),
    layers.MaxPooling2D(),
    layers.Conv2D(n_filter, s_filter, padding='same', activation=actv),
    layers.MaxPooling2D(),
    layers.Dropout(dropout),
    layers.Flatten(),
    layers.Dense(n_neurons, activation=actv),
    layers.Dense(output_classes)

    ])
  model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
  # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  #model.fit(x_train, y_train, epochs=epochs)
  model.fit(x_train, y_train, epochs=epochs,validation_data=(x_val,y_val),callbacks=[WandbCallback()])

  #history.history['accuracy']
  # print(history.history.keys())

In [ ]:
cnv(16,3,'relu',128,1,0.2,'yes')

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
!pip install wandb

In [30]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: hindanjali (use `wandb login --relogin` to force relogin)


True

In [39]:
sweep_config={
    'method':'random',
    'metrix' :
    {'name':'accuracy',
    'goal' : 'maximize'
  },
  'parameters':{
   'n_filter':{'values':[16,32,64]},
   'dropout':{'values':[0.2,0.3]},
   'data_aug':{'values':['yes','no']},
   'batch_norm':{'values':['yes','no']},
   'learning_rate':{'values':[0.0001,0.001] },
   'epochs':{'values':[3,5,7,10]},
   'activation_f':{'values':['tanh','relu']},
   'n_neurons':{'values':[32,64,128]},
   's_filter':{'values':[3,5,10]},
   'weight_decay':{'values':[0.0005,0.5]},
   #'accuracy': 'accuracy'
   #'accuracy':'acc'
  }
}


In [40]:
def a1():
  config_defaults={
  'epochs':5,
  'batch_size':16,
  'learning_rate':0.0001,
  'dropout':0.2,
  'data_aug':'yes',
  'batch_norm':'yes',
  'n_filter':32,
  'n_neurons':128,
  's_filter':3,
  'activation_f':'relu'
  }

  run=wandb.init(config=config_defaults)

  config=wandb.config

  cnv(config.n_filter,config.s_filter,config.activation_f,config.n_neurons,config.epochs,config.dropout,config.data_aug)

  #model.summary()model.summary()
  #model.summary()
  #print(model.summary())
  #model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

  #model.fit(x_train, y_train, epochs=config.epochs,validation_data=(x_val,y_val),callbacks=[WandbCallback()])



In [ ]:
a1

In [41]:
sweep_id = wandb.sweep(sweep_config,project="Assign2_part_a")

Create sweep with ID: qy1vvmd6
Sweep URL: https://wandb.ai/hindanjali/Assign2_part_a/sweeps/qy1vvmd6


In [ ]:
wandb.agent(sweep_id,a1)

wandb: Agent Starting Run: ydr1jgyh with config:
wandb: 	activation_f: tanh
wandb: 	batch_norm: yes
wandb: 	data_aug: yes
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	n_filter: 32
wandb: 	n_neurons: 32
wandb: 	s_filter: 5
wandb: 	weight_decay: 0.0005


Epoch 1/5
282/282 [==============================] - 253s 891ms/step - loss: 2.2457 - accuracy: 0.1633 - val_loss: 2.0985 - val_accuracy: 0.2690
Epoch 2/5
282/282 [==============================] - 254s 899ms/step - loss: 2.1103 - accuracy: 0.2406 - val_loss: 2.1026 - val_accuracy: 0.2670
Epoch 3/5
282/282 [==============================] - 252s 894ms/step - loss: 2.0564 - accuracy: 0.2655 - val_loss: 2.1387 - val_accuracy: 0.2640
Epoch 4/5
282/282 [==============================] - 253s 896ms/step - loss: 2.0289 - accuracy: 0.2653 - val_loss: 2.0089 - val_accuracy: 0.2800
Epoch 5/5
282/282 [==============================] - 253s 897ms/step - loss: 1.9899 - accuracy: 0.2813 - val_loss: 1.9737 - val_accuracy: 0.3130


epoch,4
loss,1.99012
accuracy,0.28659
val_loss,1.97374
val_accuracy,0.313
_runtime,1272
_timestamp,1618085587
_step,4
best_val_loss,1.97374
best_epoch,4


epoch,▁▃▅▆█
loss,█▄▃▂▁
accuracy,▁▅▆▇█
val_loss,▆▆█▂▁
val_accuracy,▂▁▁▃█
_runtime,▁▃▅▆█
_timestamp,▁▃▅▆█
_step,▁▃▅▆█


wandb: Agent Starting Run: vm1hwk1b with config:
wandb: 	activation_f: relU
wandb: 	batch_norm: no
wandb: 	data_aug: no
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0001
wandb: 	n_filter: 64
wandb: 	n_neurons: 32
wandb: 	s_filter: 5
wandb: 	weight_decay: 0.5


...

epoch,▁▂▃▅▆▇█
loss,█▅▄▃▃▂▁
accuracy,▁▄▅▆▇▇█
val_loss,▆▇█▃▂▁▁
val_accuracy,▂▃▁▇▆██
_runtime,▁▂▃▅▆▇█
_timestamp,▁▂▃▅▆▇█
_step,▁▂▃▅▆▇█


wandb: Agent Starting Run: d6yozxk4 with config:
wandb: 	activation_f: relU
wandb: 	batch_norm: yes
wandb: 	data_aug: no
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0001
wandb: 	n_filter: 16
wandb: 	n_neurons: 32
wandb: 	s_filter: 5
wandb: 	weight_decay: 0.5


Run d6yozxk4 errored: ValueError('Unknown activation function: relU')
wandb: ERROR Run d6yozxk4 errored: ValueError('Unknown activation function: relU')
wandb: Agent Starting Run: agfdv4ad with config:
wandb: 	activation_f: relU
wandb: 	batch_norm: no
wandb: 	data_aug: yes
wandb: 	dropout: 0.2
wandb: 	epochs: 7
wandb: 	learning_rate: 0.001
wandb: 	n_filter: 16
wandb: 	n_neurons: 128
wandb: 	s_filter: 10
wandb: 	weight_decay: 0.0005


Run agfdv4ad errored: ValueError('Unknown activation function: relU')
wandb: ERROR Run agfdv4ad errored: ValueError('Unknown activation function: relU')
wandb: Sweep Agent: Waiting for job.
500 response executing GraphQL.
{"errors":[{"message":"Post \"http://anaconda.default.svc.cluster.local/search\": context deadline exceeded","path":["agentHeartbeat"]}],"data":{"agentHeartbeat":null}}
wandb: ERROR Error while calling W&B API: Post "http://anaconda.default.svc.cluster.local/search": context deadline exceeded (<Response [500]>)
wandb: Job received.
wandb: Agent Starting Run: ovmsbygu with config:
wandb: 	activation_f: tanh
wandb: 	batch_norm: yes
wandb: 	data_aug: yes
wandb: 	dropout: 0.3
wandb: 	epochs: 7
wandb: 	learning_rate: 0.001
wandb: 	n_filter: 32
wandb: 	n_neurons: 128
wandb: 	s_filter: 5
wandb: 	weight_decay: 0.5


Epoch 1/7
184/282 [==================>...........] - ETA: 1:24 - loss: 2.2644 - accuracy: 0.1540